In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_teddynote.messages import stream_response

import time

# Ollama 모델을 불러옵니다.
llm = ChatOllama(model="gemma3:12b")

# 프롬프트
prompt = ChatPromptTemplate.from_template("{topic} 에 대하여 간략히 설명해 줘.")

# 질의 내용
summarize_prompt = f"""
다음은 한 매장의 최근 운영 데이터를 요약한 내용입니다.
아래 [데이터]를 기반으로 매장의 현재 상황을 분석하고, 문제가 있다면 그 원인을 추론해 주세요.
또한 아래 [요청사항]을 기반으로 매장의 개선을 위한 구체적인 전략도 제안해 주세요.

영어로 답변해주세요.

[데이터]

일평균 방문 고객 수: 320명 → 최근 일주일간 270명으로 감소

평균 고객 체류 시간: 25분

점심 시간(12시~2시) 매출: 일 평균 대비 35% 감소

최근 한 달간 리뷰 평점: 4.5점 → 3.8점

직원 2명 퇴사 → 대체 인력 미배치


[요청사항]

위 데이터를 기반으로 매장의 현재 상황을 진단해 주세요.

어떤 문제가 발생하고 있는지, 그 원인이 무엇인지 추론해 주세요.

이를 개선하기 위한 실행 전략을 제안해 주세요.
"""

start_time = time.time()

# response = llm.invoke(prompt)
response = llm.stream(summarize_prompt)
stream_response(response)

end_time = time.time()

In [ ]:
# 1-2. LLM 모델이 이전 대화를 기억하도록 하기

from langchain_teddynote.messages import stream_response
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI

# 메모리 생성 (기록 저장용)
memory = ConversationBufferMemory(memory_key="chat_history", input_key="user_query")

# 질의 내용
pinky_prompt = """
당신은 레스토랑을 돌아다니는 '핑키'라는 이름을 가진 모바일 로봇입니다.
지금까지의 대화는 다음과 같습니다 : {chat_history}

당신의 임무는 레스토랑 오너의 {user_query}에 대해 {chat_history}를 기반으로 친절하고 진실되게 답변하는 것입니다.
또한 당신은 {user_query}에 대해 2~3줄 이내로 짧고 간결하게 답변하여야 합니다.
"""

pinky_template = PromptTemplate(template = pinky_prompt, input_variables=["chat_history", "user_query"])

# Ollama 모델을 불러옵니다.
llm = ChatOllama(model="gemma3:12b")

chain = LLMChain(
    llm=llm,
    prompt=pinky_template,
    memory=memory,
    verbose=True
)